In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras import regularizers, models, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import optuna
import wandb
import gc
import cv2

import os
import sqlite3

In [ ]:
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("TensorFlow is using the GPU \n", gpus)
else:
    print("No GPU detected.")
    
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
from wandb.integration.keras import WandbMetricsLogger

wandb.require("core")
wandb.login()

In [ ]:
# Se cargan los datos
# Se debe de realizar una transformación en los datos, convirtiéndolos de -1/1 a 0/1:


ds = pd.read_csv("/tf/Face-Recognition/CelebA/list_attr_celeba.txt", sep = r"\s+", skiprows = 1)
ds.iloc[:, 0:39] = (ds.iloc[:, 0:39] == 1).astype("int32")
ds

In [ ]:
img_dir = "/img_align_celeba"

ds["image_path"] = ds.index.map(lambda x: os.path.join(img_dir, x))
ds.reset_index(inplace = True)
ds.rename(columns = {"index" : "image"}, inplace = True)
ds.head()

In [ ]:
df_split = pd.read_csv(
    "/tf/Face-Recognition/CelebA/Eval/list_eval_partition.txt", 
    sep = r"\s+",
    names = ["image", "partition"])

In [ ]:
ds = ds.merge(df_split, on = "image")
ds.head()

In [ ]:
df_train = ds[ds["partition"] == 0]
df_val   = ds[ds["partition"] == 1]
df_test  = ds[ds["partition"] == 2]

In [ ]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (128, 128))
    img = img / 255.0
    return img, label

def make_dataset(paths, labels, batch_size = 32, shuffle = False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
train_ds = make_dataset(
    df_train["image_path"].values, 
    df_train.iloc[:, 1:41].values,
    shuffle = True
)

val_ds = make_dataset(
    df_val["image_path"].values,
    df_val.iloc[:, 1:41].values,
    shuffle = False
)

test_ds = make_dataset(
    df_test["image_path"].values,
    df_test.iloc[:, 1:41].values,
    shuffle = False
)

In [ ]:
df_train_small = df_train.sample(frac = 0.05, random_state = 5)
df_val_small = df_val.sample(frac = 0.05, random_state = 5)
df_test_small = df_test.sample(frac = 0.05, random_state = 5)

train_ds_small = make_dataset(
    df_train_small["image_path"].values,
    df_train_small.iloc[:, 1:41].values,
    shuffle = True,
)

val_ds_small = make_dataset(
    df_val_small["image_path"].values,
    df_val_small.iloc[:, 1:41].values,
)

test_ds_small = make_dataset(
    df_test_small["image_path"].values,
    df_test_small.iloc[:, 1:41].values,
)

In [ ]:
import glob

print("JPEG files:", len(glob.glob("/img_align_celeba/*.jpg")))
print("PNG files:", len(glob.glob("/img_align_celeba/*.png")))

In [ ]:
# Función para los bloques residuales

def residual_block(x, kernel, kernel_size, activation, dropout, dropout_rate, regularizer, r_2):
        
    residual = x  
        
    if dropout == "y":
        # Camino "principal"
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same",
                          activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
        
        # Capa intermedia Dropot
        x = layers.Dropout(dropout_rate)(x)
            
        # Capa lineal
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same",
                          kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
            
    else: 
        # Camino "principal"
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same",
                          activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
                
        # Capa lineal
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same",
                          kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)

    # Suma de la conexión residual
    x = layers.add([x, residual]) 
    x = layers.Activation(activation)(x)
        
    return x

In [ ]:
def objective(trial):

    tf.keras.backend.clear_session()

    inputs = layers.Input(shape = (256, 256, 3))
    
    #############################################################################################################
    
    # Optuna sugiere función de activación para todas las capas
    activation = trial.suggest_categorical(f"activation_L{i+2}", ["relu", "relu6", "leaky_relu"])
    
    # Optuna sugiere regularizador
    regularizer = "L2"
    r_2 = 0.0 # trial.suggest_float("regularizer_value_2", 1e-7, 1e-5, log = True)
    
    # Optuna sugiere el número de capas
    n_layers = 4 # trial.suggest_int("N_layers", 15,20)
    
    # Optuna sugiere número de kernels y su tamaño en la primer capa convolucional
    
    kernel_1 = 8
    size_1 = 2
    
    # Optuna sugiere Learning Rate y Optimizador
    
    lr = trial.suggest_float("learning_rate", 2.5e-4, 1e-3, log = True)
    
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop"])
    
                              
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
                              
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr)               
    
    #############################################################################################################
    
    # Primera convolución
    
    x = layers.Conv2D(kernel_1, (size_1,size_1), padding = "same")(inputs)
    x = layers.Activation(activation)(x)
    x = layers.BatchNormalization()(x)
    
    kernel_per_layer = [kernel_1]
    kernel_size_per_layer = [size_1]
    
    # Optuna sugiere número de kernels su tamaño y función de activación; también sugiere Dropout
    # y regularizadores
    
    dropout_per_layer = []
    dropout_percentage_per_layer = []
    
    n_kernel =  16
    
    kernel_size = 3
    
    kernel_size_per_layer.append(kernel_size)
    dropping_out = "n" # trial.suggest_categorical("Dropout", ["y", "n"])
    
    for i in range(n_layers):
                              
        dropout = trial.suggest_categorical(f"Dropout_L{i+2}", ["y", "n"])
        dropout_per_layer.append(dropout)
                              
        dropout_rate = trial.suggest_float(f"Dropout_value_L{i+2}",0.1, 0.15)
        
        # Capa Convolucional i-ésima:
        
        # Se elige entre Dropout o un Regularizador
        
        if dropping_out == "y":
            
            if dropout == "y":
            
                dropout_percentage_per_layer.append(dropout_rate)

                ker = int(n_kernel*2**(i))
                kernel_per_layer.append(ker)

                x = layers.Conv2D(ker, (kernel_size, kernel_size), strides = 2, padding = "same",
                                  activation = activation)(x)
                x = layers.BatchNormalization()(x)
                x = residual_block(x, ker, kernel_size, activation, dropout, dropout_rate, regularizer, r_2)
                
                number = trial.suggest_int(f"Layers_{i}", 1, 2)
                
                for k in range (number): 
                    ker2 = ker*(1**k)
                    
                    x = layers.Conv2D(ker2, (kernel_size, kernel_size), padding = "same",
                                      activation = activation)(x)
                    x = layers.BatchNormalization()(x)
                    x = residual_block(x, ker2, kernel_size, activation, dropout, dropout_rate, regularizer, r_2)
            
            else:
                
                dropout_percentage_per_layer.append(0.0)

                ker = int(n_kernel*2**(i))
                kernel_per_layer.append(ker)
                
                number = trial.suggest_int(f"Layers_{i}", 1, 2)

                x = layers.Conv2D(ker, (kernel_size, kernel_size), strides = 2, padding = "same",
                                  activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
                x = layers.BatchNormalization()(x)
                x = residual_block(x, ker, kernel_size, activation, dropout, dropout_rate, regularizer, r_2)
                
                for k in range (number): 
                    ker2 = ker*(1**k)
                
                    x = layers.Conv2D(ker2, (kernel_size, kernel_size), padding = "same",
                                      activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
                    x = layers.BatchNormalization()(x)
                    x = residual_block(x, ker2, kernel_size, activation, dropout, dropout_rate, regularizer, r_2)
            
        else:
            dropout_percentage_per_layer.append(0.0)

            ker = int(n_kernel*2**(i))
            kernel_per_layer.append(ker)

            x = layers.Conv2D(ker, (kernel_size, kernel_size), strides = 2, padding = "same",
                              activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
            x = layers.BatchNormalization()(x)
            x = residual_block(x, ker, kernel_size, activation, "n", dropout_rate, regularizer, r_2)
            
            number = trial.suggest_int(f"Layers_{i}", 1, 2)
            
            for k in range (number):
                
                ker2 = ker*(1**k)
                
                x = layers.Conv2D(ker2, (kernel_size, kernel_size), padding = "same",
                                  activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
                x = layers.BatchNormalization()(x)
                x = residual_block(x, ker2, kernel_size, activation, "n", dropout_rate, regularizer, r_2)
            
            x = layers.Dropout(0.10)(x)
            
            
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
                              
    outputs = layers.Dense(4, activation = "softmax", dtype = "float32")(x)
        
    model = models.Model(inputs, outputs)
                              
    model.compile(optimizer = optimizer,
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])
    
    #############################################################################################################

    wandb.init(
        project = "Conv2D-Residual-Trials-ExpSeries1.0",
        name = f"Trial_{trial.number}",
        reinit = True,
        config = {
            "kernel_1": kernel_1,
            "size_1": size_1,
            "activation": activation,
            "n_layers": n_layers,
            "n_kernel": n_kernel,
            "kernel_size": kernel_size,
            "kernel_per_layer": kernel_per_layer,
            "kernel_size_per_layer": kernel_size_per_layer,
            "regularizer": regularizer,
            "r_value2": r_2,
            "Dropout": dropping_out, 
            "dropout_per_layer": dropout_per_layer,
            "dropout_percentage_per_layer": dropout_percentage_per_layer,
            "learning_rate": lr,
            "optimizer": activation,
        }
    )
    
    #############################################################################################################
    
    """
    Callbacks
    """
    early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 10, restore_best_weights = True)
    lr_reduction = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 7)
#     tensorboard_cb = TensorBoard(log_dir = "/workspace/Optuna-Trials/Plant-Pathology-Classificator/tf_debug", histogram_freq = 1, write_graph = True,
#                                  write_images = False)
    
    #############################################################################################################
    
    """
    Creación del modelo
    """
    
    try:
        print(model.summary())
    
        history = model.fit(
            train_ds_small, 
            validation_data = val_ds_small,
            epochs = 200,
            verbose = 1, 
            callbacks = [WandbMetricsLogger(log_freq = 5), early_stopping, lr_reduction]
        )

        val_loss = min(history.history["val_loss"])
        val_accuracy = max(history.history["val_accuracy"])
        
        train_loss = min(history.history["loss"])
        train_accuracy = max(history.history["accuracy"])
    
    except tf.errors.ResourceExhaustedError as e:
        
        print(f"Intento {trial.number} falló debido a: {e}")
        
        tf.keras.backend.clear_session()
        wandb.finish()
        gc.collect()
        
        return float("inf")

    except Exception as e:
        
        print(f"Intento {trial.number} falló. Unexpected error: {e}")
        
        tf.keras.backend.clear_session()
        wandb.finish()
        gc.collect()
        
        return float("inf")
    
    # score = val_loss + 0.1 * (train_loss - val_loss)
    
    score = val_accuracy
    
    # score = train_loss 
    
    tf.keras.backend.clear_session()
    gc.collect()
    wandb.finish()

    return 1-score

In [ ]:
study = optuna.create_study(
    study_name = "Conv2D-Residual-Trials-ExpSeries1.0",
    direction = "minimize",
    storage = "sqlite:////workspace/Optuna-Trials/Conv2D-Residual-Trials-ExpSeries1.0_study.db",
    load_if_exists = True
)

study.optimize(objective, n_trials = 500)